In [1]:
import librosa
import scipy.fft
import pywt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors


def analyze_audio(file_path, segment_length=30, sample_rate=1000):
    """
    Analyzes a segment of an audio file using Fourier and Wavelet Transforms.

    :param file_path: Path to the audio file.
    :param segment_length: Length of the audio segment to analyze, in seconds.
    :param sample_rate: Sampling rate for downsampling the audio.
    """
    # Load the audio file with librosa (downsampling for simplicity)
    audio, sr = librosa.load(file_path, sr=sample_rate)

    # Convert to mono
    audio_mono = librosa.to_mono(audio)

    # Calculate the number of samples to consider for the given segment length
    num_samples = segment_length * sample_rate

    # Extract the desired segment
    audio_segment = audio_mono[:num_samples]

    # Fourier Transform
    fft_audio = scipy.fft.fft(audio_segment)
    fft_freq = scipy.fft.fftfreq(len(audio_segment), 1/sample_rate)
    # Keep only positive frequencies
    fft_audio = np.abs(fft_audio)[:len(audio_segment)//2]
    # Adjust frequency array accordingly
    fft_freq = fft_freq[:len(audio_segment)//2]

    # Continuous Wavelet Transform (CWT)
    # Using logarithmic scale for widths based on the Fourier Transform results
    max_freq = sample_rate // 2
    widths = np.logspace(0, np.log10(max_freq), num=50)
    cwt_audio, _ = pywt.cwt(audio_segment, widths, 'mexh')

    # Normalize CWT for better visualization using logarithmic scaling
    cwt_audio_abs_log = np.log1p(np.abs(cwt_audio))
    vmin, vmax = np.percentile(cwt_audio_abs_log, [5, 95])

    # Discrete Wavelet Transform (DWT)
    coeffs = pywt.wavedec(audio_segment, 'db1', level=4)
    dwt_audio = pywt.waverec(coeffs, 'db1')

    # Time vector for plotting
    t = np.linspace(0, len(audio_segment)/sample_rate, num=len(audio_segment))

    # # Plotting
    # plt.figure(figsize=(15, 10))

    # # Original Audio Signal
    # plt.subplot(4, 1, 1)
    # plt.plot(t, audio_segment)
    # plt.title('Original Audio Signal')
    # plt.xlabel('Time [s]')
    # plt.ylabel('Amplitude')

    # # Fourier Transform (Positive Frequencies Only)
    # plt.subplot(4, 1, 2)
    # plt.plot(fft_freq, fft_audio)
    # plt.title('Fourier Transform (Positive Frequencies)')
    # plt.xlabel('Frequency [Hz]')
    # plt.ylabel('Amplitude')

    # # Continuous Wavelet Transform with logarithmic color scale
    # plt.subplot(4, 1, 3)
    # norm = colors.Normalize(vmin=vmin, vmax=vmax)
    # plt.imshow(cwt_audio_abs_log, extent=[0, len(
    #     audio_segment)/sample_rate, 1, max_freq], cmap='viridis', aspect='auto', norm=norm)
    # plt.title('Continuous Wavelet Transform - Log Scale')
    # plt.xlabel('Time [s]')
    # plt.ylabel('Frequency [Hz]')

    # # Discrete Wavelet Transform (Reconstructed Signal)
    # plt.subplot(4, 1, 4)
    # plt.plot(t, dwt_audio)
    # plt.title('Discrete Wavelet Transform (Reconstructed Signal)')
    # plt.xlabel('Time [s]')
    # plt.ylabel('Amplitude')

    # plt.tight_layout()
    # plt.show()


# Usage
# Replace with the actual path to your audio file
file_path = './Guilty Hero.mp3'
analyze_audio(file_path, segment_length=30, sample_rate=1000)

: 